# 1. 라이브러리 호출

In [1]:
import pandas as pd
import re
import kss
import krwordrank
from konlpy.tag import Kkma
from konlpy.tag import Okt
from pykospacing import Spacing
from krwordrank.sentence import summarize_with_sentences
from tqdm import tqdm
import time


kkma = Kkma()
okt = Okt()
spacing = Spacing()

# 2. 전처리된 블로그 리뷰 불러오기

In [4]:
df = pd.read_csv('processed_place_blog_review3.csv', encoding='utf-8-sig')

In [5]:
df

,index,num,name,blog or review,blog name,like_count_list,command_count_list,img_count,neighber_count,blog review
0,0,1,배곧 또잇카페 배곧점,블로그 리뷰,TTOEATCAFE 배곧카페 뷰맛집!,5,6,21,1071,하기에 산책 지는 모습 보며 공원 걷는 좋은지 산책 야외 테라스 음료 마시기 가져가...
1,1,2,배곧 또잇카페 배곧점,블로그 리뷰,배곧 한울공원 뷰 [또잇카페],7,4,23,191,케이 쿠 맛집 또 잇 카페 배곧 09:0021 10:0021 : 00 10:0020...
2,2,3,배곧 또잇카페 배곧점,블로그 리뷰,배곧카페 TTOEATCAFE 또잇 : 배곧한울공원카페 / 배곧디저트,5,2,30,111,공원 가까이 카페 배곧 카페 스타트 배곧 디저트 배곧 카페 또 잇 카페 배곧 엠 플...
3,3,4,배곧 또잇카페 배곧점,블로그 리뷰,"시흥 배곧카페 또잇카페, 정왕동 신상 디저트 카페",22,18,22,547,뿌시러 카페 전체 트인 오션 한눈 볼 한껏 분위기 왔답니다 또 잇 카페 031804...
4,4,5,배곧 또잇카페 배곧점,블로그 리뷰,시흥 배곧 카페 추천: 또잇카페,7,1,16,311,오빠 잔소리 한 카페 찾았네 배곧 카페 식물 예쁘게 꾸며놓은 인테리어 인상 깊었고 ...
...,...,...,...,...,...,...,...,...,...,...
2789,2856,86,배곧 Cafe 다합,블로그 리뷰,시흥 배곧브런치카페 다합 배곧 데이트하고 브런치 먹고~,20,4,29,1227,곳도 맛있느 많아서 하기에 배곧 다 다 브런치 데이트 배곧 카페 다 합 브런치 카페...
2790,2857,87,배곧 Cafe 다합,블로그 리뷰,배곧브런치카페! 다합에서 브런치 먹은후기:),4,2,36,637,애정 배곧 배곧 다 합 마카롱 가려다가 카페 였어요 그때 카페 토닉 시켜서 먹었거든...
2791,2858,88,배곧 Cafe 다합,블로그 리뷰,브런치맛집 다합,1,3,23,17,아기자기한 분위기 더욱 좋았어요 . 화분 어찌나 이쁘던 사장 쎈스 돋보 엿 어요 ....
2792,2859,1,배곧 카페바이마스,블로그 리뷰,cafe ByMars,19,6,16,616,애플 참고 아이맥 못 참아서 컷 스피커 다르긴 깔끔하고 아기자기한 인테리어 소품 좋...


In [6]:
reviews = df['blog review']

In [7]:
reviews

0       하기에 산책 지는 모습 보며 공원 걷는 좋은지 산책 야외 테라스 음료 마시기 가져가...
1       케이 쿠 맛집 또 잇 카페 배곧 09:0021 10:0021 : 00 10:0020...
2       공원 가까이 카페 배곧 카페 스타트 배곧 디저트 배곧 카페 또 잇 카페 배곧 엠 플...
3       뿌시러 카페 전체 트인 오션 한눈 볼 한껏 분위기 왔답니다 또 잇 카페 031804...
4       오빠 잔소리 한 카페 찾았네 배곧 카페 식물 예쁘게 꾸며놓은 인테리어 인상 깊었고 ...
                              ...                        
2789    곳도 맛있느 많아서 하기에 배곧 다 다 브런치 데이트 배곧 카페 다 합 브런치 카페...
2790    애정 배곧 배곧 다 합 마카롱 가려다가 카페 였어요 그때 카페 토닉 시켜서 먹었거든...
2791    아기자기한 분위기 더욱 좋았어요 . 화분 어찌나 이쁘던 사장 쎈스 돋보 엿 어요 ....
2792    애플 참고 아이맥 못 참아서 컷 스피커 다르긴 깔끔하고 아기자기한 인테리어 소품 좋...
2793    61 280 01092308889 11:0022 : 12:00 매주 가능 쌀쌀하더니...
Name: blog review, Length: 2794, dtype: object

# 3. 블로그 리뷰 전처리

In [8]:
corpus = []

for i in tqdm(range(0, len(reviews)), desc='전처리 진행율'):
    review = okt.pos(str(reviews[i]), stem=True) # 형태소 분석, 품사태깅
    word = []
    for j in review:
        if not j[1] == 'Noun': # tagging이 명사가 아니면
            continue #건너뛰기
        if len(j[0])==1: # 한 글자짜리는 조사일 수 있으니 건너뛰기
            continue
        word.append(j[0]) # 나머지들을 append
    word = ' '.join(word) # 띄어쓰기로 join
    word += '.' # 끝에 . 붙이기
    corpus.append(word)
    


전처리 진행율: 100%|███████████████████████████████████████████████████████████████| 2794/2794 [00:58<00:00, 48.13it/s]


In [9]:
corpus

['산책 모습 공원 산책 야외 테라스 음료 마시기 카페 배곧 매장 계시 위생 배곧 카페 매장 출입 명부 안심 손소독제 카페 메뉴판 카페 사장 매니저 바리스타 디플로마 자격 소유 전문가 시더 고요 더욱 아메리카노 라떼 특징 카페 시간 캐모마일 레몬 그라스 페퍼민트 홍차 블렌딩 티백 상품 카페 케익 케익 종류 마카롱 종류 별로 쫀득 마카롱 필링 가득 배곧 카페 카페 아침 카페 커피 아침 카페 또한 판매 매장 구경해 매장 화이트 인테리어 요한 공원 공원 커피 한잔 배곧 카페 카페 회의 마련 단체 테이블 초록 초록 나무 인테리어 배곧 카페 음료 진동 배곧 카페 포토 마련 사진 요시키 노랑 레몬 더욱 음료 상큼 디저트 음료 가시 매장 수제 레몬 상큼 레몬 입안 아주 바깥 초록 초록 풍경 마시기 주스 아주 망고 주스 느낌 주스 말캉 느낌 망고 주스 아이 더욱 망고 과육 바나나 바나나 디저트 바나나 플람베 바나나 플람베 음료 케이크 정말 커피 배곧 카페 카페 공원 산책 걷기 연인 공원 산책 분위기 카페 시간 공원 맛집 배곧 맛집 배곧 맛집 맛집 아메리카노 맛집 인테리어 맛집 배곧 디저트 배곧 카페 디저트 배곧 카페.',
 '케이 맛집 카페 배곧 바깥 좌석 내부 개별 좌석 단체 좌석 카페 카페 음료 무디 등등 크루아상 케이크 마카롱 대박 케이크 카페 아메리카노 라떼 깔루 마끼 케이크 생크림 케이크 메뉴 딸기 라떼 아메리카노 산미 케이크 라떼 루아 마끼 케이크 케이 마카롱 호호 개구리 바닐라 라이언 오레오 정도 보이 거울 맛집 케이크 맛집 배곧 카페.',
 '공원 가까이 카페 배곧 카페 스타트 배곧 디저트 배곧 카페 카페 배곧 플러스 테크노 사무실 플러스 지식 산업 센터 위치 카페 직장인 배곧 카페 매장 분위기 의자 테라스 글라스 테이블 인테리어 경쟁 배곧 카페 카페 공원 해질녘 배곧 카페 분위기 테리 배곧 식물 카페 플렌 테리 어란 거구 매장 내부 카페 분위기 물씬 눈사람 글구 전신 거울 포토 전신 맛집 배곧 카페 카페 디저트 코너 직행 스콘 맛집 대만 음료 함뷰 배곧 디저

# 4. 키워드 추출 데이터 추가

In [11]:
df.insert(10, 'blog keyword', corpus) # group by 써서 식당과 묶어 주려고 insert

In [12]:
df

,index,num,name,blog or review,blog name,like_count_list,command_count_list,img_count,neighber_count,blog review,blog keyword
0,0,1,배곧 또잇카페 배곧점,블로그 리뷰,TTOEATCAFE 배곧카페 뷰맛집!,5,6,21,1071,하기에 산책 지는 모습 보며 공원 걷는 좋은지 산책 야외 테라스 음료 마시기 가져가...,산책 모습 공원 산책 야외 테라스 음료 마시기 카페 배곧 매장 계시 위생 배곧 카페...
1,1,2,배곧 또잇카페 배곧점,블로그 리뷰,배곧 한울공원 뷰 [또잇카페],7,4,23,191,케이 쿠 맛집 또 잇 카페 배곧 09:0021 10:0021 : 00 10:0020...,케이 맛집 카페 배곧 바깥 좌석 내부 개별 좌석 단체 좌석 카페 카페 음료 무디 등...
2,2,3,배곧 또잇카페 배곧점,블로그 리뷰,배곧카페 TTOEATCAFE 또잇 : 배곧한울공원카페 / 배곧디저트,5,2,30,111,공원 가까이 카페 배곧 카페 스타트 배곧 디저트 배곧 카페 또 잇 카페 배곧 엠 플...,공원 가까이 카페 배곧 카페 스타트 배곧 디저트 배곧 카페 카페 배곧 플러스 테크노...
3,3,4,배곧 또잇카페 배곧점,블로그 리뷰,"시흥 배곧카페 또잇카페, 정왕동 신상 디저트 카페",22,18,22,547,뿌시러 카페 전체 트인 오션 한눈 볼 한껏 분위기 왔답니다 또 잇 카페 031804...,뿌시러 카페 전체 오션 한눈 분위기 카페 공휴일 맞은편 주차 배곧 플러스 지식 산업...
4,4,5,배곧 또잇카페 배곧점,블로그 리뷰,시흥 배곧 카페 추천: 또잇카페,7,1,16,311,오빠 잔소리 한 카페 찾았네 배곧 카페 식물 예쁘게 꾸며놓은 인테리어 인상 깊었고 ...,오빠 잔소리 카페 배곧 카페 식물 인테리어 인상 매장 햇볕 매장 범상 카페 입구 앞...
...,...,...,...,...,...,...,...,...,...,...,...
2789,2856,86,배곧 Cafe 다합,블로그 리뷰,시흥 배곧브런치카페 다합 배곧 데이트하고 브런치 먹고~,20,4,29,1227,곳도 맛있느 많아서 하기에 배곧 다 다 브런치 데이트 배곧 카페 다 합 브런치 카페...,곳도 배곧 브런치 데이트 배곧 카페 브런치 카페 브런치 카페 배곧 카페 우리 카페 ...
2790,2857,87,배곧 Cafe 다합,블로그 리뷰,배곧브런치카페! 다합에서 브런치 먹은후기:),4,2,36,637,애정 배곧 배곧 다 합 마카롱 가려다가 카페 였어요 그때 카페 토닉 시켜서 먹었거든...,애정 배곧 배곧 마카롱 카페 그때 카페 토닉 라서 분위기 무척 가라 카페 브런치 배...
2791,2858,88,배곧 Cafe 다합,블로그 리뷰,브런치맛집 다합,1,3,23,17,아기자기한 분위기 더욱 좋았어요 . 화분 어찌나 이쁘던 사장 쎈스 돋보 엿 어요 ....,분위기 더욱 화분 사장 쎈스 어요 디어 시점 이건 화장실 하쟌.
2792,2859,1,배곧 카페바이마스,블로그 리뷰,cafe ByMars,19,6,16,616,애플 참고 아이맥 못 참아서 컷 스피커 다르긴 깔끔하고 아기자기한 인테리어 소품 좋...,애플 참고 아이맥 스피커 인테리어 소품 디테일 라떼아트 조금 식감 조명도 플래쉬 터...


# 5. 식당과 키워드 연결

In [13]:
df2 = df.groupby('name', sort=False).agg({'blog keyword': ','.join}).reset_index() # 식당 이름으로 이어서 붙이기
df2

,name,blog keyword
0,배곧 또잇카페 배곧점,산책 모습 공원 산책 야외 테라스 음료 마시기 카페 배곧 매장 계시 위생 배곧 카페...
1,배곧 카츠오모이,카츠 모이 카츠 모이 배곧 오픈 도로 안쪽 줄줄이 쪼끔 웰던 참고 주심 카츠 모이 ...
2,배곧 양푼애등갈비,맛집 하나 갈비 양푼 갈비 안산 양푼 갈비 모임 만큼 맛집 증명 시흥 배곧 진출 시...
3,배곧 투파인드피터 배곧점,인테리어 감각 이탈리아 대비 중하 감각 인테리어 가격 부상 피터 어째 즈음 글로리 ...
4,배곧 소쇄원 배곧본점,배곧 신도시 배곧 맛집 맛집 소쇄원 배곧 신도시 맛집 소쇄원 주차장 사진 오른쪽 소...
5,배곧 정동진,사진첩 방문 창가 좌석 특대 구이왕 가리비 음료수 메뉴판 조개 구이 대왕 해물 문필...
6,배곧 Tailor.Button,카페 참고 카페 에스프레소 카페 배곧 카페 테일러 버튼 신상 카페 래서 번화가 안쪽...
7,배곧 딱이야직화두마리치킨 배곧점,치킨 직화 치킨 연락처 주문 갈릭 빠다 아이 살로 주문 쫄면 사리 사리 핫도그 아이...
8,배곧 카페 검은순수 배곧본점,취미 카페 순수 배곧 감성 카페 순수 건물 건물 배곧 주차장 무료 사장 주차 주차 ...
9,배곧 Poke all day 포케&샐러드 배곧신도시점,대체로 영양성분 균형 흐뜨러져 균형 마찬가지 균형 식단 외식 포케올 만족 식사 배곧...


In [14]:
keyword = df2['blog keyword']
name = df2['name']

In [15]:
restaurant_keyword = []
for i in keyword:
    restaurant_keyword.append(i.split('.')) # 아까 붙인 . 기준으로 split 해서 이중리스트 형태로 변환
restaurant_keyword

[['산책 모습 공원 산책 야외 테라스 음료 마시기 카페 배곧 매장 계시 위생 배곧 카페 매장 출입 명부 안심 손소독제 카페 메뉴판 카페 사장 매니저 바리스타 디플로마 자격 소유 전문가 시더 고요 더욱 아메리카노 라떼 특징 카페 시간 캐모마일 레몬 그라스 페퍼민트 홍차 블렌딩 티백 상품 카페 케익 케익 종류 마카롱 종류 별로 쫀득 마카롱 필링 가득 배곧 카페 카페 아침 카페 커피 아침 카페 또한 판매 매장 구경해 매장 화이트 인테리어 요한 공원 공원 커피 한잔 배곧 카페 카페 회의 마련 단체 테이블 초록 초록 나무 인테리어 배곧 카페 음료 진동 배곧 카페 포토 마련 사진 요시키 노랑 레몬 더욱 음료 상큼 디저트 음료 가시 매장 수제 레몬 상큼 레몬 입안 아주 바깥 초록 초록 풍경 마시기 주스 아주 망고 주스 느낌 주스 말캉 느낌 망고 주스 아이 더욱 망고 과육 바나나 바나나 디저트 바나나 플람베 바나나 플람베 음료 케이크 정말 커피 배곧 카페 카페 공원 산책 걷기 연인 공원 산책 분위기 카페 시간 공원 맛집 배곧 맛집 배곧 맛집 맛집 아메리카노 맛집 인테리어 맛집 배곧 디저트 배곧 카페 디저트 배곧 카페',
  ',케이 맛집 카페 배곧 바깥 좌석 내부 개별 좌석 단체 좌석 카페 카페 음료 무디 등등 크루아상 케이크 마카롱 대박 케이크 카페 아메리카노 라떼 깔루 마끼 케이크 생크림 케이크 메뉴 딸기 라떼 아메리카노 산미 케이크 라떼 루아 마끼 케이크 케이 마카롱 호호 개구리 바닐라 라이언 오레오 정도 보이 거울 맛집 케이크 맛집 배곧 카페',
  ',공원 가까이 카페 배곧 카페 스타트 배곧 디저트 배곧 카페 카페 배곧 플러스 테크노 사무실 플러스 지식 산업 센터 위치 카페 직장인 배곧 카페 매장 분위기 의자 테라스 글라스 테이블 인테리어 경쟁 배곧 카페 카페 공원 해질녘 배곧 카페 분위기 테리 배곧 식물 카페 플렌 테리 어란 거구 매장 내부 카페 분위기 물씬 눈사람 글구 전신 거울 포토 전신 맛집 배곧 카페 카페 디저트 코너 직행 스콘 맛집 대만 음료 함뷰 배곧

# 6. 최종 키워드 추출

In [16]:
stop = 0

for i in range(len(name)):
    print(name[i])
    test = restaurant_keyword[i]
    st=''
    stop += 1
    for j in range(len(test)):
        texts = test[j]
        st+= texts
        texts = st.split('. ')
    try:
        stopwords = {'진짜', '정말', '맛집', '카페', '테이블', '배곧', '주문', '모이', '오픈', '추가',
                    '피터', '리조', '최고', '직원', '다음', '서비', '가성', '데이', '다른'}
        # keywords = KR-WordRank로 학습된 키워드와 랭크 값이 dict형태로, sents = 핵심 문장이 list of str 형식
        keywords, sents = summarize_with_sentences(
                                            texts, #문자열
                                            stopwords = stopwords, #키워드에서 제거할 단어
                                            num_keywords=100, # 키워드로 추출될 키워드 개수
                                            num_keysents=10 # 핵심문장으로 추출될 문장의 개수
                                            )
    except ValueError:
        print('key 없음')
        print()
        continue
        
    for word, j in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]: # 키워드 랭킹 순으로 sort
        print('#%s' % word)
    print()
    if stop == len(name): 
        break

배곧 또잇카페 배곧점
#스콘
#케이크
#커피
#느낌
#공원
#음료
#디저트
#인테리어
#라떼
#마카롱

배곧 카츠오모이
#카츠
#돈까스
#소금
#등심
#안심
#카레
#트러플
#사장
#시간
#드레싱

배곧 양푼애등갈비
#갈비
#버섯
#양푼
#고기
#양념
#콩나물
#사리
#메밀
#당면
#하나

배곧 투파인드피터 배곧점
#파스타
#메뉴
#분위기
#크림
#스테이크
#그린
#전복
#매콤
#조명
#소스

배곧 소쇄원 배곧본점
#고기
#소쇄원
#소고기
#메뉴
#세트
#육회
#부위
#차돌
#미인
#계란찜

배곧 정동진
#오이도
#조개
#해물
#칼국수
#정동진
#구이
#라면
#치즈
#파전
#가리비

배곧 Tailor.Button
#커피
#버튼
#원두
#라떼
#테일러
#작품
#공간
#느낌
#갤러리
#에스

배곧 딱이야직화두마리치킨 배곧점
#치킨
#직화
#마리
#배달
#양념
#메뉴
#빠다
#매콤
#치즈
#소스

배곧 카페 검은순수 배곧본점
#순수
#메뉴
#음료
#라떼
#사장
#크림
#시흥
#아이스
#와플
#서비스

배곧 Poke all day 포케&샐러드 배곧신도시점
#포케
#샐러드
#소스
#토핑
#다이어트
#메밀
#메뉴
#인테리어
#점심
#식단

배곧 구십도씨
#도씨
#아이스
#치즈
#메뉴
#커피
#브라운
#음료
#인테리어
#식물
#라떼

배곧 엣홈파스타
#파스타
#뇨끼
#라자냐
#예약
#소스
#생면
#메뉴
#오일
#버섯
#풍기

배곧 세유카츠
#카츠
#돈까스
#돈카츠
#치즈
#소금
#소스
#카레
#새우젓
#주산
#트러플

배곧 모미지
#마카
#미지
#초밥
#참치
#스시
#하나
#전복
#식감
#전갱이
#사케

배곧 저스트텐동 배곧신도시점
#텐동
#튀김
#저스트
#소스
#메뉴
#단호박
#수란
#유자
#베니스
#샐러드

배곧 윌로우 우드
#티라미수
#우드
#로우
#라떼
#분위기
#커피
#인테리어
#느낌
#크런치
#아메리카노

배곧 왈라비스스테이크
#스테이크
#왈라비
#파스타
#메뉴
#분위기
#스프
#샐러드
#와인
#오일
#부라타

배곧 판다소바
#소바
#마제
#다소
#식초
#토핑
